In [ ]:
from selenium import webdriver
import pandas as pd
import numpy as np
import time

# Login to fb @news page
def login(driver, account, password): 
    
    # Enter account
    driver.find_element_by_xpath("//*[@id=\"m_login_email\"]").send_keys(account)

    # Enter password
    driver.find_element_by_xpath("//*[@id=\"m_login_password\"]").send_keys(password)

    # Submit login
    driver.find_element_by_xpath("//*[@id=\"u_0_5\"]").click()
    time.sleep(2)
    
    # Proceed to home page
    driver.find_element_by_xpath('//*[@id="root"]/div[1]/div/div/div[3]/div[1]/div/div/a').click()
    time.sleep(2)

# Get to article comment section
def toCommentSection(page_id, driver, article_id):
    # Generate url to each article comment section
    url_article = 'https://mobile.facebook.com/story.php?story_fbid=' + article_id + '&id=' + page_id + '&fs=5&focus_composer=0&ref=page_internal'
    driver.get(url_article)

# Get link to actual article site @article page
def getArticleLink(driver, article_index):
    try: 
        article_link = driver.find_element_by_css_selector("a[class=\"touchable _4qxt\"]").get_attribute("href")
        return article_link
    except:
        return ''

# View more comments @article page
def moreComments(driver):
    try:
        more_btn = driver.find_element_by_css_selector("a[class=\"_108_\"]")
        last_height = driver.execute_script("return document.body.scrollHeight")
        is_more = True
    except:
        is_more = False
    while is_more:
        try: 
            more_btn.click()
            time.sleep(1.5)
            current_height = driver.execute_script("return document.body.scrollHeight")
            if current_height == last_height:
                is_more = False
            else:
                last_height = current_height
        except:
            is_more = False

# Get comment and likes @each comment webelement
def getCommentLikes(comment, article_index):
    
    # init return values
    likes = '0'
    has_sticker = 0
    has_gif = 0
    has_img = 0
    has_vid = 0
    num_friend_tags = 0
    num_hashtags = 0
    friend_text = []
    hashtag_text = []
    aria_text = ""
    comment_text = ""
    
    # Get likes
    try:
        likes = comment.find_element_by_css_selector("span[class=\"_14va\"]").text
    except:
        likes = '0'
    
    # Catch sticker
    try:
        comment.find_element_by_css_selector("i[class=\"img _2b1w img _2sxw\"]")
        has_sticker = 1
    except:
        # Catch GIF
        try:
            comment.find_element_by_css_selector("div[class=\"_14vb\"]")
            has_gif = 1 
        except:
            # Catch img
            try:
                img = comment.find_element_by_css_selector("div[class=\"_2b1t attachment\"]")
                has_img = 1
                # Get img description
                aria_text = img.find_element_by_css_selector("i[role=\"img\"]").get_attribute("aria-label")
            except:
                pass

                # Catch video
                try:
                    comment.find_element_by_css_selector("img[class=\"_34wo img\"]")
                    has_vid = 1
                except:
                    pass
                
            # Get content
            try:
                body = comment.find_element_by_css_selector("div[data-sigil=\"comment-body\"]")
                
                # Catch comments without text
                try:
                    body.find_element_by_css_selector("a[href*=\"fref=nf\"]")
                except:
                    # Get comment text
                    comment_text = body.text
                    # Catch tags (friend_tags and hashtags)
                    try:
                        tags = body.find_elements_by_css_selector("a[href*=\"refid=52\"]")
                        friend_text = [friend.text for friend in tags]
                        try:
                            hashtags = body.find_elements_by_css_selector("a[href*=\"hashtag\"]")
                            hashtag_text = [hashtag.text for hashtag in hashtags]
                            for text in hashtag_text:
                                friend_text.remove(text)
                            num_hashtags = len(hashtags)
                            comment_text = replaceStr(comment_text, hashtag_text)
                        except:
                            pass
                        num_friend_tags = len(tags) - num_hashtags
                        if num_friend_tags != 0:
                            comment_text = replaceStr(comment_text, friend_text)
                    except:
                        pass
            except:
                pass
            
    # Change list to tuple to prevent numpy conversion ValueError
    hashtag_text = ','.join(hashtag_text) 
    
    # Return results as tuples
    res_tuple = (article_index, comment_text,
                 has_sticker, has_gif, has_img, has_vid, 
                 num_friend_tags, num_hashtags, hashtag_text, aria_text, likes)
    return res_tuple

def getStatusLikes(driver, article_index):
    headline = ''
    status = ''
    has_headline = 0
    has_vid = 0
    has_img = 0
    has_status = 0
    likes = '0'
    
    try: 
        headline = driver.find_element_by_css_selector("span[data-ad-preview=\"headline\"]").text
        has_headline = 1
    except:
        headline = ''
        try:
            driver.find_element_by_css_selector("i[data-ad-preview=\"video-cover\"]")
            has_vid = 1
        except:
            pass
        try:
            driver.find_element_by_css_selector("i[data-sigil=\"photo-image\"]")
            has_img = 1
        except:
            pass
    
    try:
        status = driver.find_element_by_css_selector("div[data-ad-preview=\"message\"]").text
        has_status = 1
    except:
        pass
    
    try:
        likes = driver.find_element_by_css_selector("div[class=\"_1g06\"]").text
    except:
        pass
    
    res_tuple = (article_index, headline, status, has_headline, has_status, has_img, has_vid, likes)
    return res_tuple


def scrapeAll(page, page_id, driver, article_ids, start_idx):
    initCSV(page)
    print('init .csv successful!')
    article_links = []
    comments = []
    posts = []
    for i in range(start_idx,len(article_ids)):
        try:
            toCommentSection(page_id, driver, article_ids[i])
            posts.append(getStatusLikes(driver, i))
            article_links.append((i, article_ids[i], getArticleLink(driver,i)))
            moreComments(driver)
            try: 
                comments_ele = driver.find_elements_by_css_selector('div[data-sigil="comment"]')
                for comment in comments_ele:
                    try:
                        comments.append(getCommentLikes(comment, i))
                    except:
                        pass
            except:
                pass
        except Exception as e:
            print(e)
        
        # Save results for every 20 articles
        if (i%20 == 0) and (i!=0):
            try:
                appendResults(page, article_links, comments, posts)
                print('up to article %i saved!' %i)
            except Exception as e:
                print('------------------------------------------------')
                print('Something went WRONG saving articles %i to %i.' %(i-19, i))
                print(e)
                print('------------------------------------------------')
            # Clear results 
            article_links = []
            comments = []
            posts = []
    
    # Save final remaining results
    try: 
        appendResults(page, article_links, comments, posts)
        print('last few results saved.')
    except:
        pass
        
    print('all articles processed.')

def initCSV(page):
    import csv
    
    with open('results/'+page+'_article_links.csv', 'w', encoding="utf-8") as csvfile:
        filewriter = csv.writer(csvfile, delimiter=',',
                                quotechar='|', quoting=csv.QUOTE_MINIMAL)
        filewriter.writerow(['article_index', 'article_id', 'article_links'])
        
    with open('results/'+page+'_comments.csv', 'w', encoding="utf-8") as csvfile:
        filewriter = csv.writer(csvfile, delimiter=',',
                                quotechar='|', quoting=csv.QUOTE_MINIMAL)
        filewriter.writerow(['article_index', 'comment_text', 'has_sticker', 
                            'has_gif', 'has_img', 'has_vid', 'num_friend_tags',
                            'num_hashtags', 'hashtag_text', 'aria_text', 'likes'])
        
    with open('results/'+page+'_posts.csv', 'w', encoding="utf-8") as csvfile:
        filewriter = csv.writer(csvfile, delimiter=',',
                                quotechar='|', quoting=csv.QUOTE_MINIMAL)
        filewriter.writerow(['article_index', 'headline', 'status', 'has_headline', 
                             'has_status', 'has_img', 'has_vid', 'likes'])
    
def appendResults(page, article_links, comments, posts):
    if len(article_links) != 0:
        df_article_links = pd.DataFrame(np.array(article_links), columns=['article_index',
                                                                'article_id',
                                                                'article_links']) 
        with open('results/'+page+'_article_links.csv', 'a', encoding="utf-8") as f:
            df_article_links.to_csv(f, header=False, index=False)
    
    if len(comments) != 0:
        df_comments = pd.DataFrame(np.array(comments), columns=['article_index', 
                                                            'comment_text',
                                                            'has_sticker',
                                                            'has_gif',
                                                            'has_img',
                                                            'has_vid',
                                                            'num_friend_tags',
                                                            'num_hashtags',
                                                            'hashtag_text',
                                                            'aria_text',
                                                            'likes'])
        
        with open('results/'+page+'_comments.csv', 'a', encoding="utf-8") as f:
            df_comments.to_csv(f, header=False, index=False)    
    
    if len(posts) != 0:
        df_posts = pd.DataFrame(np.array(posts), columns=['article_index', 
                                                    'headline', 
                                                    'status', 
                                                    'has_headline', 
                                                    'has_status', 
                                                    'has_img', 
                                                    'has_vid', 
                                                    'likes'])
    
        with open('results/'+page+'_posts.csv', 'a', encoding="utf-8") as f:
            df_posts.to_csv(f, header=False, index=False)
        
# Recursively, from a list of string, remove its items from a string
def replaceStr(text, to_remove_list, to_remove_idx=0):
    output = text.replace(to_remove_list[to_remove_idx], '')
    if to_remove_idx+1 < len(to_remove_list):
        return replaceStr(output, to_remove_list, to_remove_idx+1)
    else:
        return output
    
    
def get(page, account, password, start_idx=0):
    # Start webdriver
    driver = webdriver.Chrome()
    driver.get('https://mobile.facebook.com')
    
    # Login to fb
    login(driver, account, password)
    
    # Set path to article_ids.csv
    path_to_article_ids = 'results-1000-win-fixed\\'+page+'_ids.csv'
    
    # Set path to page_ids.csv
    path_to_page_ids = 'page_ids.csv'
    
    # Load article_ids
    article_ids = pd.read_csv(path_to_article_ids, header=None)[0]
    
    # Load page_ids
    page_ids = pd.read_csv(path_to_page_ids, header=0, index_col=0, dtype=str)
    page_id = page_ids.loc[page].get('page_id')
    
    # Start scraping, save in the process (every 20 articles)
    scrapeAll(page, page_id, driver, article_ids, start_idx)